In [1]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
import pandas as pd

c:\Users\ENVY 13TH GEN\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set padding token to EOS token
model = TFGPT2LMHeadModel.from_pretrained("gpt2")

c:\Users\ENVY 13TH GEN\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [3]:
# Define custom loss function
def custom_loss(y_true, y_pred):
    mask = tf.not_equal(y_true, -100)  # Create a mask where -100 labels are ignored
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)
    mask = tf.cast(mask, loss.dtype)  # Ensure mask is the same dtype as loss
    loss = loss * mask  # Apply mask
    return tf.reduce_sum(loss) / tf.reduce_sum(mask)  # Compute average over non-ignored tokens


In [4]:
# Compile model with the custom loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer, loss=custom_loss)

In [5]:
# Load your custom dataset
data = pd.read_csv("Conversation.csv")
conversations = [(row['question'], row['answer']) for _, row in data.iterrows()]

In [6]:
input_texts = [q + " " + a for q, a in conversations]
input_encodings = tokenizer(input_texts, padding="max_length", max_length=40, return_tensors="tf", truncation=True)

In [7]:
input_ids = input_encodings["input_ids"]
attention_masks = input_encodings["attention_mask"]

In [8]:
# Shift input_ids for labels
labels = tf.roll(input_ids, shift=-1, axis=1)
labels = tf.where(labels == tokenizer.pad_token_id, -100, labels)

In [9]:
# Create TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices(({
    "input_ids": input_ids,
    "attention_mask": attention_masks
}, labels))
dataset = dataset.shuffle(len(input_ids)).batch(8)

In [10]:
# Train the model
model.fit(dataset, epochs=3)

Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert




InvalidArgumentError: Graph execution error:

Detected at node custom_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\ENVY 13TH GEN\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\ENVY 13TH GEN\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\ENVY 13TH GEN\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\ENVY 13TH GEN\AppData\Roaming\Python\Python312\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "c:\Users\ENVY 13TH GEN\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 641, in run_forever

  File "c:\Users\ENVY 13TH GEN\AppData\Local\Programs\Python\Pytho